### Known Issues (as of 06.12.19)

This is a python3 project, therefore the sys.version output below should show python 3 as used kernel
If thats not the case refer to 
``` bash
$ jupyter kernelspec list
```
which shows where the juypter kernels are defined at. Inside the shown folders lies a kernel.json which leads to the interpreter used by jupyter. For python3 it should show something along `"argv": [ "python3", "-m",...` for sure. For python2 accordingly. 

It's important to use `pip3` as installer, as we install for python3.

pyqt5 should be installed! Therefore:
``` bash
$ pip3 install qtpy5
```

Some further dependencies are given:
``` bash
$ pip3 install pandas scipy sympy colorama dill
```

In [1]:
import sys
print (sys.version)
print(sys.executable)
sys.path.append(r'../geo-adjust')

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
/usr/bin/python3


In [2]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import sympy as sp
import matplotlib.pyplot as plt
#plt.rcParams["figure.figsize"] = (10,7)

from numpy import asarray, vstack, pi, nan, identity, mean, hstack, diag, array, int64, nanmean, concatenate, max, eye, hypot, insert
from scipy.spatial import KDTree
from scipy.signal import savgol_filter

from argparse import Namespace

from lasertracker import load_ros_log
from robot_arm import load_and_save_tf
from utils.misc import dcm2euler, euler2dcm, window_slicer

from utils.misc import format_figure
from utils.misc import sphere2cart

from geo_adjust.config import GHModelConfig
from geo_adjust.gh import GHAdjust



from IPython.display import display

# 1 Transformation Parameter and Mounting Vector

Schätzung der Stationierung und des Mounting Vektors

In [4]:
import pandas as pd
tf_f = r'data/tf_051219.csv'
win_f = r'data/tslices_051219.csv'
lt_f = r'data/lt_051219.txt'

In [5]:
from lasertracker import load_ros_log

lt_df = load_ros_log(lt_f, version=2)
lt_df.dropna(inplace=True)
lt_df['stamp'] = pd.to_datetime(lt_df['t'], unit='s')
lt_df.head()

,t,sec,mus,hz,v,d,time,y,x,z,stamp
0,1.575577e+09,0,0,-0.582061,1.568828,7.310136,00:00:00,4.018716,6.106374,0.014389,2019-12-05 20:17:45.890720367
1,1.575577e+09,0,20000,-0.582061,1.568828,7.310136,00:00:00.020000,4.018716,6.106374,0.014389,2019-12-05 20:17:45.890880346
2,1.575577e+09,0,40000,-0.582061,1.568828,7.310136,00:00:00.040000,4.018716,6.106374,0.014389,2019-12-05 20:17:45.890923738
3,1.575577e+09,0,60000,-0.582061,1.568828,7.310136,00:00:00.060000,4.018716,6.106374,0.014389,2019-12-05 20:17:45.891013861
4,1.575577e+09,0,80000,-0.582061,1.568827,7.310136,00:00:00.080000,4.018716,6.106374,0.014396,2019-12-05 20:17:45.891057491


In [6]:
lt_df['stamp'] = lt_df.loc[0, 'stamp'] + lt_df['time'] - pd.Timedelta(0.4, unit='s')
lt_df.set_index('stamp', drop=False, inplace=True)
lt_df.head()

,t,sec,mus,hz,v,d,time,y,x,z,stamp
stamp,,,,,,,,,,,
2019-12-05 20:17:45.490720367,1.575577e+09,0,0,-0.582061,1.568828,7.310136,00:00:00,4.018716,6.106374,0.014389,2019-12-05 20:17:45.490720367
2019-12-05 20:17:45.510720367,1.575577e+09,0,20000,-0.582061,1.568828,7.310136,00:00:00.020000,4.018716,6.106374,0.014389,2019-12-05 20:17:45.510720367
2019-12-05 20:17:45.530720367,1.575577e+09,0,40000,-0.582061,1.568828,7.310136,00:00:00.040000,4.018716,6.106374,0.014389,2019-12-05 20:17:45.530720367
2019-12-05 20:17:45.550720367,1.575577e+09,0,60000,-0.582061,1.568828,7.310136,00:00:00.060000,4.018716,6.106374,0.014389,2019-12-05 20:17:45.550720367
2019-12-05 20:17:45.570720367,1.575577e+09,0,80000,-0.582061,1.568827,7.310136,00:00:00.080000,4.018716,6.106374,0.014396,2019-12-05 20:17:45.570720367


In [7]:
lt_df.shape

(19576, 11)

In [8]:
from utils.misc import window_grouper

lt_df = window_grouper(lt_df, win_f, margin=0.4)

In [9]:
plt.figure()
plt.plot(lt_df.x, '+')
pts = {}
for i in lt_df.group.unique(): 
    pts[i] = lt_df[lt_df.group==i].mean()
    plt.plot(lt_df.x[lt_df.group==i], '+')
format_figure()

/home/finn/.local/lib/python3.6/site-packages/pandas/plotting/_matplotlib/converter.py:103: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


In [10]:
df_pts = pd.DataFrame.from_dict(pts, orient='index', columns=lt_df.mean().keys().tolist())
df_pts.sort_index(inplace=True)
df_pts.dropna(inplace=True)
df_pts.head()

,t,sec,mus,hz,v,d,time,y,x,z
J001,1.575577e+09,19.333333,526666.666667,-0.647967,1.508232,7.417108,00:00:19.860000,4.467959,5.902184,0.463743
J002,1.575577e+09,28.135135,504864.864865,-0.620970,1.618144,6.599570,00:00:28.640000,3.835489,5.361509,-0.312357
J003,1.575577e+09,34.796460,463539.823009,-0.621076,1.634213,6.541499,00:00:35.260000,3.798917,5.309197,-0.414560
J004,1.575577e+09,42.567568,472432.432432,-0.620819,1.608988,6.703480,00:00:43.040000,3.896582,5.448650,-0.255954
J005,1.575577e+09,49.818182,471818.181818,-0.602786,1.625653,6.601965,00:00:50.290000,3.737285,5.430251,-0.361981


In [11]:
rob_df = load_and_save_tf(tf_f)

In [12]:
df_pts_ur = window_slicer(rob_df, win_f, margin=0.4)

In [13]:
df_pts_ur.head()

,time,x,y,z,qx,qy,qz,qw,ut,k,...,dcm_2_var,dcm_3_var,dcm_4_var,dcm_5_var,dcm_6_var,dcm_7_var,dcm_8_var,ro_var,pi_var,ya_var
J001,1.575577e+09,-0.110488,0.095229,0.989273,0.000122,-0.000279,-0.708235,0.705977,1.575577e+09,1802.0,...,2.741370e-10,8.351103e-15,8.086687e-10,8.522666e-10,8.524830e-10,2.738634e-10,1.212217e-16,2.738635e-10,8.524830e-10,8.087368e-10
J002,1.575577e+09,-0.910513,-0.112076,0.208493,0.001205,0.001003,-0.706275,0.707936,1.575577e+09,2900.0,...,1.049580e-09,6.468961e-15,6.600565e-10,2.608534e-10,2.606239e-10,1.049417e-09,2.708037e-15,1.049427e-09,2.606265e-10,6.596974e-10
J003,1.575577e+09,-0.939061,-0.112055,0.149969,0.184183,-0.181884,-0.682465,0.683547,1.575577e+09,3727.5,...,5.779255e-10,6.416910e-15,6.352013e-10,4.254225e-10,3.185422e-10,5.781023e-10,1.927857e-10,7.708914e-10,3.185446e-10,7.422031e-10
J004,1.575577e+09,-0.856526,-0.112106,0.244728,-0.181931,0.183892,-0.681989,0.684088,1.575577e+09,4701.0,...,8.089976e-10,7.522459e-15,6.937886e-10,3.757616e-10,2.811263e-10,8.096129e-10,2.693094e-10,1.078932e-09,2.811297e-10,7.888577e-10
J005,1.575577e+09,-0.910370,-0.170339,0.184046,0.271257,0.271814,-0.652179,0.653605,1.575577e+09,5607.0,...,1.017249e-09,1.465178e-10,5.355340e-10,1.450571e-10,1.450017e-10,1.891001e-09,1.466677e-10,3.803812e-09,2.916772e-10,2.834700e-09


In [14]:
#df_pts_ur = pd.read_pickle(tf_f.replace('.csv', '.pk'))
plt.figure()
plt.subplot(121)
plt.plot(df_pts_ur.x, df_pts_ur.y, '+')
plt.axis('equal')
plt.title('UR')
plt.subplot(122)
plt.plot(df_pts.x, df_pts.y, '+')
plt.axis('equal')
plt.title('LT')
format_figure()

In [15]:
params = {
    'atr_sigma': 20e-6,
    'var_rob': 0.00030 ** 2,  # robot position accuracy
    'var_rob_angle': (0.011 / 200 * pi) ** 2,  # robot pose angle accuracy
    'scale': True
}
p = Namespace(**params)

In [16]:
if p.scale:
    gh_config = GHModelConfig.load(r'data/models/lt_trafo_ccr_scale.pk')
else:
    gh_config = GHModelConfig.load(r'data/models/lt_trafo_ccr.pk')

solver = GHAdjust(gh_config, max_iterations=30)

# Initial parameters
if p.scale:
    x0 = [0., 0., -50. / 200 * pi, 0., -5.25, -6.25, -0.6, 0., 0., 0.]
else:
    x0 = [0., 0., -50. / 200 * pi, -5.25, -6.25, -0.6, 0., 0., 0.]
    
solver.set_initial_params(x0)

In [17]:
gh_config.get_model_syms()

(Matrix([
 [-e + ty + (m + 1)*(sin(pru)*sin(rru)*cos(yru) - sin(yru)*cos(rru))*(mv_x*sin(yeu)*cos(peu) + mv_y*(sin(peu)*sin(reu)*sin(yeu) + cos(reu)*cos(yeu)) + mv_z*(sin(peu)*sin(yeu)*cos(reu) - sin(reu)*cos(yeu)) + rob_y) + (m + 1)*(sin(pru)*cos(rru)*cos(yru) + sin(rru)*sin(yru))*(-mv_x*sin(peu) + mv_y*sin(reu)*cos(peu) + mv_z*cos(peu)*cos(reu) + rob_z) + (m + 1)*(mv_x*cos(peu)*cos(yeu) + mv_y*(sin(peu)*sin(reu)*cos(yeu) - sin(yeu)*cos(reu)) + mv_z*(sin(peu)*cos(reu)*cos(yeu) + sin(reu)*sin(yeu)) + rob_x)*cos(pru)*cos(yru)],
 [-n + tx + (m + 1)*(sin(pru)*sin(rru)*sin(yru) + cos(rru)*cos(yru))*(mv_x*sin(yeu)*cos(peu) + mv_y*(sin(peu)*sin(reu)*sin(yeu) + cos(reu)*cos(yeu)) + mv_z*(sin(peu)*sin(yeu)*cos(reu) - sin(reu)*cos(yeu)) + rob_y) + (m + 1)*(sin(pru)*sin(yru)*cos(rru) - sin(rru)*cos(yru))*(-mv_x*sin(peu) + mv_y*sin(reu)*cos(peu) + mv_z*cos(peu)*cos(reu) + rob_z) + (m + 1)*(mv_x*cos(peu)*cos(yeu) + mv_y*(sin(peu)*sin(reu)*cos(yeu) - sin(yeu)*cos(reu)) + mv_z*(sin(peu)*cos(reu)*cos

In [18]:
# remove outliers
for i in ['J006']:
    df_pts.drop(i, inplace=True, errors='ignore')
    df_pts_ur.drop(i, inplace=True, errors='ignore')

obs = hstack((df_pts_ur[['x', 'y', 'z', 'ro', 'pi', 'ya']].values, df_pts[['x', 'y', 'z']].values))
obs.shape[0]

45

In [19]:
sigma_ll = diag([p.var_rob] * obs.size)

for i in range(df_pts.shape[0]):
    sigma_ll[3 + i * 9:6 + i * 9, 3+i * 9:6 + i * 9] = diag([p.var_rob_angle] * 3)
    sigma_ll[6 + i * 9:9 + i * 9, 6 + i * 9:9 + i * 9] = diag([p.atr_sigma ** 2] * 3)

solver.add_data(obs, sigma_ll, var0=nanmean(diag(sigma_ll)),
                row_names=df_pts.index.tolist())

In [20]:
result = solver.solve()

In [21]:
x_sym, l_sym = gh_config.get_syms()
x_sym

[rru, pru, yru, m, ty, tx, tz, mv_x, mv_y, mv_z]

In [22]:
if p.scale:
    result.parameters.set_params_config(tuple(x_sym),
                                        ['gon'] * 3 + ['ppm'] + ['m'] * 3 + ['mm'] * 3,
                                        ['r', 'p', 'y', 'm', 'te', 'tn', 'th', 'mvx', 'mvy', 'mvz'],
                                        cxs=[lambda x: x / pi * 200] * 3 + [lambda x: x * 1e6] + [lambda x: x] * 3 + [lambda x: x * 1e3] * 3,
                                        formats=['.5f'] * 3 + ['.1f'] + ['.5f'] * 3 + ['.2f'] * 3)
else:
    result.parameters.set_params_config(tuple(x_sym),
                                        ['gon'] * 3 + ['m'] * 3 + ['mm'] * 3,
                                        ['r', 'p', 'y', 'te', 'tn', 'th', 'mvx', 'mvy', 'mvz'],
                                        cxs=[lambda x: x / pi * 200] * 3 + [lambda x: x] * 3 + [
                                            lambda x: x * 1e3] * 3,
                                        formats=['.5f'] * 6 + ['.2f'] * 3)

result.observations.set_obs_config(tuple(l_sym),
                                   ['mm'] * 3 + ['gon'] * 3 + ['mm'] * 3,
                                   ['lt_x', 'lt_y', 'lt_z', 'ro', 'pi', 'ya', 'rob_x', 'rob_y', 'rob_z'],
                                   cxs=[lambda x: x * 1e3] * 3 + [lambda x: x / pi * 200] * 3 + [
                                       lambda x: x * 1e3] * 3,
                                   formats=['.3f'] * 3 + ['.4f'] * 3 + ['.2f'] * 3,
                                   groups=['Robot Position']*3 + ['Robot Angle']*3 + ['Laser Tracker Position']*3)

In [23]:
result.model.print_summary()
result.parameters.print(print_correlation=False)
result.parameters.plot_corr()


 Result Summary 
 - Convergence: 				  [ OK]
    ├ # Iterations: 			    13
    └ Criterium: 					  1.000e-12
 - Hauptprobe: 				  [ OK]
    └ Max Eps: 					  -6.661e-16
 - Globaltest: 				  [NOK]
    ├ Test: 						  eq 
    ├ Var Prio: 					  0.000
    ├ Var Post: 					  0.000
    ├ Ratio:    					  0.490
    ├ Alpha: 						  0.100
    └ Testquantity: 				  0.801 < 0.490 < 1.217
 - Residuals:
    ├ SSE (vTv):                0.000
    ├ MSE:                      0.000
    ├ RMSE:                     0.000
    ├ WSSE (vTPv):              0.000
    ├ WMSE:                     0.000
    └ WRMSE:                    0.000



 Paramters
 - Parameter Estimates ± σ:
    ├  rru:     -0.02384 ±  0.00377   [gon]
    ├  pru:      0.40321 ±  0.00335   [gon]
    ├  yru:     38.84492 ±  0.00268   [gon]
    ├    m:        370.9 ±     41.2   [ppm]
    ├   ty:      6.09552 ±  0.00009   [m]
    ├   tx:      4.48510 ±  0.00007   [m]
    ├   tz:     -0.61399 ±  0.00004   [m]
    ├  mvₓ:        -0.51 ±     

In [ ]:
obs = result.observations.dataframe()

In [ ]:
obs['Robot Position']['nv_max []'] = obs['Robot Position'][['nv_robₓ []', 'nv_rob_y []', 'nv_rob_z []']].max(axis=1)

In [ ]:
obs['Robot Position'].sort_values('nv_max []', ascending=False).head()

In [ ]:
obs['Robot Position'].sort_values('nv_rob_y []', ascending=False).head()
# obs['Robot Angle'].sort_values('nv_peu []', ascending=False).head()
# obs['Laser Tracker Position'].sort_values('nv_u []', ascending=False).head()